In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import math
from mpl_toolkits.mplot3d import Axes3D
import random
import matplotlib.colors as mcolors
import tensorflow as tf
from tensorflow.keras import layers,models
from keras.layers import BatchNormalization, Dropout

In [ ]:
#make the imges
rainfall_data = pd.read_csv('input.csv')
rainfall_data.drop(['INDEX','YEAR','TOT_Cal'],axis=1,inplace= True)
rainfall_data.fillna(0,inplace= True)
rainfall_data = rainfall_data[rainfall_data['TOTRF'] != 0]
rainfall_data.reset_index(drop=True, inplace=True)

In [ ]:
rainfall_data

In [ ]:
def make_img(df):
    season_mapping = {
    1: 1, 2: 1, 3: 1,
    4: 2, 5: 2, 6: 2,
    7: 3, 8: 3, 9: 3,
    10: 4, 11: 4, 12: 4
    }
    df['Season'] = df['MN'].map(season_mapping)
    df.drop(['MN', 'DT', 'DAY'],axis=1, inplace=True)
    image_set= np.zeros((len(df),40,int(df.iloc[:,-2].max()+1)),dtype= float)
    image_level= np.zeros(len(df),dtype= float)
    hour= -1
    for col_label, col in df.items():
        if hour <= 23:
            hour= hour+1
            for index, value in col.items():
                image_set[index,hour,int(value)]=1
        else:
            for index,row in df.iterrows():
                image_set[index,-int(2 * row['Season'])::,:]=1
                image_level[index]= row['TOTRF']
    return image_level,image_set

In [ ]:
        image_level, image_set = make_img(rainfall_data)
        model = models.Sequential()
        model.add(layers.Conv1D(32, kernel_size=5, activation='relu', input_shape=(image_set.shape[1], image_set.shape[2])))
        model.add(layers.MaxPooling1D(pool_size=2))
        model.add(layers.Conv1D(16, kernel_size=3, activation='relu', input_shape=(image_set.shape[1], image_set.shape[2])))
        model.add(layers.MaxPooling1D(pool_size=2))
        model.add(layers.Flatten())
        model.add(layers.Dense(16, activation='relu'))
        model.add(layers.Dense(16, activation='relu'))
        model.add(layers.Dense(1, activation='linear'))  # 1 neuron for regression output
        model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
        
        model.fit(image_set,image_level, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Assuming y_true contains the true total rainfall values and y_pred contains the predicted values
y_true = np.array(image_level)  # Replace with your actual values
y_pred = model.predict(image_set)  # Assuming model is your trained CNN model

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_true, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_true, y_pred)
print(f'Mean Squared Error (MSE): {mse}')
relative_error = mae / np.mean(image_level) * 100
print(f'Relative Error: {relative_error}%')
print(f'Thus accuracy: {100 - relative_error}%')

In [ ]:
image_set